#<center>Python [Pandas](http://pandas.pydata.org/pandas-docs/stable/) Cheat Sheet</center>

As a data analyst, these are common tasks I perform using Pandas

- [Pandas display output options](#options)
- [Sorting More Than One Columns](#sorting)
- [Removing duplicates](#removing_duplicates)
- [Creating a column based on values from another column](#map)
- [Replacing values in a series](#replacing_values)
- [Reading CSV or Excel files](http://pandas.pydata.org/pandas-docs/stable/io.html)
- [Renaming axis](#renaming_axis)
- [Renaming indexes or columns](#renaming_indexes_columns)
- [Binning data into intervals](#binning)
- [Grouping data](#group)
- [Sub-selecting or Slicing a DataFrame](#slicing)
- [Boolean indexing](#boolean)
- [Obtaining columns with partial column labels](#partial_column_labels)
- [Getting value counts](#value_counts)
- [Pivot table](#pivot_table)
- [Converting index to a column](#index2column)
- [How to add or fill in missing dates](#missing_dates)
- [How to connect to an ODBC data source (Windows)](#database)
- [How to convert data in wide format to long format using melt()](#melt)
- [How to convert data in long format data to wide format using pivot()](#pivot)
- [Using category data type to control sort order](#category)
- [Merging 2 dataframes using merge()](#merge)

<a id='options'></a>

### Setting display output [options](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.set_option.html)

[[back to top](#top)]

In [1]:
import pandas as pd
pd.set_option("display.max_rows",1000)
pd.set_option("display.max_columns",20)
pd.set_option('precision',7)
pd.set_option('large_repr', 'truncate')

ImportError: No module named 'pandas'